In [1]:
import os
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import argparse
from model.classifier import VGG16
from model.evaluator import FaceNet
from model.SAC import Agent
from model.gan import Generator
from utils.mi_attack import inversion_dcgan, inversion_stylegan
from utils.file_utils import load_my_state_dict, low2high, seed_everything
import pickle
import time
import warnings

In [2]:
MODEL_NAME = "VGG16"
MAX_EPISODES = 100000
MAX_STEP = 1
SEED = 42
ALPHA = 0
N_CLASSES = 1000
Z_DIM = 100
N_TARGET = 10

### load if generator should be stylegan2 trained on faces (ffhq)

In [3]:
print(f"Target Model: {MODEL_NAME}")
with open ('../checkpoints/gan/GeneratorStyleffhq.pkl', 'rb') as f:
    G = pickle.load(f)['G_ema'].cuda()
G.eval()
# G = Generator(Z_DIM)
# path_G = "checkpoints/gan/Genen.tar"
# G = torch.nn.DataParallel(G).cuda()
# G.load_state_dict(torch.load(path_G)['state_dict'], strict=False)

Target Model: VGG16


Generator(
  (synthesis): SynthesisNetwork(
    (b4): SynthesisBlock(
      (conv1): SynthesisLayer(
        (affine): FullyConnectedLayer()
      )
      (torgb): ToRGBLayer(
        (affine): FullyConnectedLayer()
      )
    )
    (b8): SynthesisBlock(
      (conv0): SynthesisLayer(
        (affine): FullyConnectedLayer()
      )
      (conv1): SynthesisLayer(
        (affine): FullyConnectedLayer()
      )
      (torgb): ToRGBLayer(
        (affine): FullyConnectedLayer()
      )
    )
    (b16): SynthesisBlock(
      (conv0): SynthesisLayer(
        (affine): FullyConnectedLayer()
      )
      (conv1): SynthesisLayer(
        (affine): FullyConnectedLayer()
      )
      (torgb): ToRGBLayer(
        (affine): FullyConnectedLayer()
      )
    )
    (b32): SynthesisBlock(
      (conv0): SynthesisLayer(
        (affine): FullyConnectedLayer()
      )
      (conv1): SynthesisLayer(
        (affine): FullyConnectedLayer()
      )
      (torgb): ToRGBLayer(
        (affine): FullyConn

### load target model

In [4]:
T = VGG16(N_CLASSES)
path_T = "../checkpoints/VGG16/nn_VGG16celeba.tar"
T = torch.nn.DataParallel(T).cuda()
T.load_state_dict(torch.load(path_T)['state_dict'], strict=False)
T.eval()

/home/bot/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/bot/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


DataParallel(
  (module): VGG16(
    (feature): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU(inplace=True)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): ReLU(inplace=True)
      (13): MaxPool2d(kernel_size=2, stride=2, padding=

### load evaluator model

In [5]:
# E = FaceNet(N_CLASSES)
# path_E = '../checkpoints/VGG16/Evalceleba.tar'
# E = torch.nn.DataParallel(E).cuda()
# E.load_state_dict(torch.load(path_E)['state_dict'], strict=False)
# E.eval()

E = VGG16(N_CLASSES)
path_E = "../checkpoints/VGG16/nn_VGG16celeba.tar"
E = torch.nn.DataParallel(T).cuda()
E.load_state_dict(torch.load(path_E)['state_dict'], strict=False)
E.eval()

DataParallel(
  (module): DataParallel(
    (module): VGG16(
      (feature): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
        (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (9): ReLU(inplace=True)
        (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (12): ReLU(inplace=True)

In [6]:
seed_everything(seed=SEED)

In [7]:
total = 0
cnt = 0
cnt5 = 0

identities = range(N_CLASSES)
# targets = identities[401:401+N_TARGET]
targets = [0]
print(len(targets))
os.makedirs("../attack_results/reinforcment_celebaStyle/images", exist_ok=True)
os.makedirs("../attack_results/reinforcment_celebaStyle/models", exist_ok=True)

1


### Attack Loop

In [8]:
for i in targets:
        start = time.time()
        agent = Agent(state_size=G.z_dim, action_size=G.z_dim, random_seed=SEED, hidden_size=256, action_prior="uniform")
        recon_image = inversion_stylegan(agent, G, T, ALPHA, z_dim=G.z_dim, max_episodes=MAX_EPISODES, max_step=MAX_STEP, label=i, model_name=MODEL_NAME)
        # output= E(low2high(recon_image))
        output = E(recon_image)
        eval_prob = F.softmax(output[1], dim=1)
        top_idx = torch.argmax(eval_prob)
        _, top5_idx = torch.topk(eval_prob, 5)

        total += 1
        if top_idx == i:
            cnt += 1
        if i in top5_idx:
            cnt5 += 1

        acc = cnt / total
        acc5 = cnt5 / total
        print("Classes {}/{}, Accuracy : {:.3f}, Top-5 Accuracy : {:.3f}".format(total, N_TARGET, acc, acc5))
        end = time.time()
        total_time = end-start
        print(f"duration for {MAX_EPISODES} episodes: {total_time:.2f}")

Target Label : 0
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Episodes 10000/100000, Confidence score for the target model : 0.0254
Episodes 20000/100000, Confidence score for the target model : 0.2517
Episodes 30000/100000, Confidence score for the target model : 0.2517
Episodes 40000/100000, Confidence score for the target model : 0.3175
Episodes 50000/100000, Confidence score for the target model : 0.5795
Episodes 60000/100000, Confidence score for the target model : 0.5795
Episodes 70000/100000, Confidence score for the target model : 0.9785
Episodes 80000/100000, Confidence score for the target model : 0.9844
Episodes 90000/100000, Confidence score for the target model : 0.9844
Episodes 100000/100000, Confidence score for the target model : 0.9844
Classes 1/10, Accuracy : 1.000, Top-5 Accuracy : 1.000
duration for 100000 episodes: 8516.75
